In [2]:
import numpy as np
import tensorflow as tf
import keras
from keras.utils import np_utils
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential
from keras import losses
from keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils import class_weight
from sklearn.metrics import recall_score, confusion_matrix, multilabel_confusion_matrix, precision_score, precision_recall_curve, average_precision_score, make_scorer
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from pandas import DataFrame
import xlsxwriter
import time

#Define Load embed
def load_embed(ind):
    doc_embed   = tf.constant(np.load(f"embeds/doc_embed_{ind}.npy"))
    word_embed  = tf.constant(np.load(f"embeds/word_embed_{ind}.npy"))
    topic_embed = tf.constant(np.load(f"embeds/topic_embed_{ind}.npy"))
    return (doc_embed, word_embed, topic_embed)

#Define Topic Dist
def topic_dist(ind):
    embeds = load_embed(ind)
    doc_embed = embeds[0]
    topic_embed = embeds[2]
    norm_doc = tf.nn.softmax(doc_embed)
    topic_dist = tf.linalg.matmul(norm_doc, topic_embed)
    return topic_dist

In [3]:
#Load Data
data = topic_dist(0)
print(data.shape)
#Remove last 10 rows (aka 2017. Cant predict 2018 without data)
inputs = data[:-10, :]
print(inputs.shape)
#Remove first 10 rows (aka 2008. Cant predict to 2008 without prev data)
outputs = data[10:, :]
print(outputs.shape)

(100, 300)
(90, 300)
(90, 300)


In [4]:
string = ""
for x in range(0,100):
    if(x % 10 != 0):
        string+=str(x)
        string+=","
print(string)

1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,78,79,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99,


In [5]:
# Function to create model, required for KerasRegressor
def create_model(layer1_size=300, layer2_size=300):
    # Create model
    model = Sequential()
    model.add(Dense(layer1_size, input_dim=300, activation='tanh'))
    model.add(Dense(layer2_size, activation='tanh'))
    model.add(Dense(300, activation='tanh'))
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


def iterator():
    fold0 = [1,2,3,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,29,31,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,58,59,61,62,63,64,65,66,67,68,69,71,72,73,74,75,76,77,78,79,81,82,83,84,85,86,87,88,89]
    fold0_test = [0,10,20,30,40,50,60,70,80]
    
    
    print("fold0",len(fold0))
    print("fold0_test",len(fold0_test))
    
    
    return [(fold0,fold0_test)]


custom_cv = iterator()

fold0 81
fold0_test 9


In [6]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# create model
model = KerasRegressor(build_fn=create_model)
# define the grid search parameters
batch_size = [10, 15, 20, 25, 30, 35]
epochs = [50, 100, 150]
hidden_size1 = [250, 275, 300, 325, 350]
hidden_size2 = [250, 275, 300, 325, 350]

param_grid = dict(batch_size=batch_size, epochs=epochs,  layer1_size=hidden_size1, layer2_size=hidden_size2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=custom_cv)
grid_result = grid.fit(inputs,outputs)


#For scoring I can try a weighted accuracy. Or a combination of Recall and PPV.
#Weighted accuracy - (Sensitivty tp/(tp+fn) + Specificity tn/(tn+fp))/2 => This is true
#An iterable yielding (train, test) splits as arrays of indices.
#To work around setting class weights, I can duplicate the minority class points in each fold only on the training data AFTER smote.
#Max amount of compute nodes I can request is 64. Shoot for 16-32 cores. Request more than 4 hours.

<ipython-input-6-b239878eb9f8>:6: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasRegressor(build_fn=create_model)


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(grid_result.cv_results_)
df

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))